In [5]:
%load_ext autoreload
%autoreload 2

import os, sys
import torch

sys.path.insert(0, '../src')
sys.path.insert(0, '..')
import utils
from models.config import Config, IMAGE_TOKEN_IDS
from main import get_model

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
default_config_path = "../configs/models/qwen/TaskSensitivity/Qwen2-VL-7B-Instruct-TaskSensitivity-Default.yaml"
overwrite_configs = True
prompts = [
    "What are the (x, y) coordinates of this person's left eye?",
    "What is the color of this person's clothes?",
    "What is the person's hair texture?",
    "What is the name of this hairstyle?",
    "Where was this photo taken?",
    "Who is this person?",
    "What country is this person from?",
    "What does (or did) this person do for a living?",
    "Who is this person's mother?",
    "What is this person's country most known for?"
]

config = utils.read_file(default_config_path)

config_paths = [default_config_path]
# Make config files
for idx, prompt in enumerate(prompts):
    config['prompt'] = prompt
    config['output_db'] = config['output_db'].replace("Default", "Prompt_{}".format(idx+1))
    save_path = default_config_path.replace("Default", "Prompt_{}".format(idx+1))
    utils.write_file(config, save_path, overwrite=overwrite_configs)
    config_paths.append(save_path)

print(len(config_paths)) # 1 default + 10 prompts

Saved file to ../configs/models/qwen/TaskSensitivity/Qwen2-VL-7B-Instruct-TaskSensitivity-Prompt_1.yaml
Saved file to ../configs/models/qwen/TaskSensitivity/Qwen2-VL-7B-Instruct-TaskSensitivity-Prompt_2.yaml
Saved file to ../configs/models/qwen/TaskSensitivity/Qwen2-VL-7B-Instruct-TaskSensitivity-Prompt_3.yaml
Saved file to ../configs/models/qwen/TaskSensitivity/Qwen2-VL-7B-Instruct-TaskSensitivity-Prompt_4.yaml
Saved file to ../configs/models/qwen/TaskSensitivity/Qwen2-VL-7B-Instruct-TaskSensitivity-Prompt_5.yaml
Saved file to ../configs/models/qwen/TaskSensitivity/Qwen2-VL-7B-Instruct-TaskSensitivity-Prompt_6.yaml
Saved file to ../configs/models/qwen/TaskSensitivity/Qwen2-VL-7B-Instruct-TaskSensitivity-Prompt_7.yaml
Saved file to ../configs/models/qwen/TaskSensitivity/Qwen2-VL-7B-Instruct-TaskSensitivity-Prompt_8.yaml
Saved file to ../configs/models/qwen/TaskSensitivity/Qwen2-VL-7B-Instruct-TaskSensitivity-Prompt_9.yaml
Saved file to ../configs/models/qwen/TaskSensitivity/Qwen2-VL-7B

In [6]:
for config_path in config_paths:
    sys.argv = ['notebooks/task_sensitivity.ipynb',
            '--config', config_path]
    config = Config()
    model = get_model(config.architecture, config)

    n_modules = 0
    layer_names = []
    for name, module in model.model.named_modules():
        if model.config.matches_module(name):
            print(name)
            layer_names.append(name)
            n_modules += 1
    utils.informal_log("{} modules matched".format(n_modules))
    break

ValueError: Device set to cuda but no GPU found for this machine

In [7]:
print(torch.cuda.is_available())

False
